In [7]:
from pathlib import Path
import zipfile
import toml
import argparse
import json


def create_unique_file(name: str) -> None:
    """Create a file given a name, and add a number if exists.
    
    """ 

    file = Path(name)
    stem: str = file.stem
    extension: str = file.suffix

    counter: int = 1
    while file.exists():
        file = Path(stem+str(counter)+extension)
        counter += 1

    file.touch()

    return file

def get_mod_details(jar_path):
    with zipfile.ZipFile(jar_path, 'r') as jar:
        try:
            with jar.open('META-INF/neoforge.mods.toml') as f:
                content = f.read().decode('utf-8')  # decode binary to string
                mods_toml = toml.loads(content)     # parse from string
                mods = mods_toml.get('mods', [])
                if mods:
                    name = mods[0].get('displayName', 'Unknown Mod Name')
                    description = mods[0].get('description', 'Unknown Description')
                    return {"name": name, "description": description}
        except:
            pass

def get_modlist(folder: str) -> list[str]:
    """""" 

    mods = Path(folder)

    modlist = []
    for file in mods.iterdir():
        if file.is_file():
            modlist.append(get_mod_details(file))

    return modlist


path = r"C:\Users\777\AppData\Roaming\ModrinthApp\profiles\Cobblemon Official Modpack [NeoForge]\mods"

modlist = get_modlist(path)

In [ ]:
from openai import OpenAI
client = OpenAI(api_key="")

In [17]:
categories = [
    "Performance Improvements",
    "Aesthetics",
    "Camera Tweaks",
    "UI Tweaks",
    "Gameplay",
    "Player Tools",
    "Mod Libraries",
    "Other"
]

schema = {
    "type": "object",
    "properties": {
        category: {
            "type": "array",
            "items": {
                "type": "string"
            }
        } for category in categories
    },
    "required": categories,
    "additionalProperties": False
}

In [28]:
response = client.responses.create(
    model="gpt-4o",  # Use the correct model
    input=[
        {"role": "system", "content": "Classify each mod into a single category. Be so sure of yourself that if you had to run this 100 times in a row you would pick the same categories each time"},
        {"role": "user", "content": json.dumps({"categories": categories, "mods": modlist})}
    ],
    text={
        "format": {
            "type": "json_schema",
            "name": "mod_classification",
            "schema": schema,
            "strict": True
        }
    }
)


classified_mods = json.loads(response.output_text)

In [29]:
with open("mofgfg.json", "w") as file:
    # Dump the JSON object into the file
    json.dump(classified_mods, file, indent=4)